In [1]:
!pip install stanza

In [2]:
# Инициализация Stanza и pandas, и также os для порционной обработки
import os
import pandas as pd
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize,pos,lemma')

2024-10-30 20:22:02 INFO: Downloaded file to /Users/alevtinashirokova/stanza_resources/resources.json
2024-10-30 20:22:02 INFO: Downloading default packages for language: ru (Russian) ...
2024-10-30 20:22:05 INFO: File exists: /Users/alevtinashirokova/stanza_resources/ru/default.zip
2024-10-30 20:22:10 INFO: Finished downloading models and saved to /Users/alevtinashirokova/stanza_resources
2024-10-30 20:22:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-10-30 20:22:10 INFO: Downloaded file to /Users/alevtinashirokova/stanza_resources/resources.json
2024-10-30 20:22:11 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |

2024-10-30 20:22:11 INFO: Using device: cpu
2024-10-30 20:22:11 INFO: Loading: tokenize
/Users/alevtinashirokova/anaconda3/lib/python3.11/site-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functi

In [3]:
# Загрузка CSV-файла
input_file_path = 'parsed_blocks.csv'
output_file_path = 'parsed_blocks_with_nlp.csv'
data = pd.read_csv(input_file_path, sep=';', on_bad_lines='skip')  # Указываем ; как разделитель

In [4]:
# Если результирующий файл существует, продолжим обработку с последнего сохранения
if os.path.exists(output_file_path):
    processed_data = pd.read_csv(output_file_path, sep=';')
else:
    # Иначе создаем новый DataFrame и добавляем новые столбцы для токенов, лемм и POS-тегов
    processed_data = data[['Meta', 'Sentence']].copy()
    processed_data['Tokens'] = None
    processed_data['Lemmas'] = None
    processed_data['POS'] = None

# Указываем размер порции
batch_size = 100  # Количество предложений на одну порцию
start_index = len(processed_data.dropna())  # Начнем с последней обработанной строки

# Цикл обработки порциями
for i in range(start_index, len(data), batch_size):
    batch = data.iloc[i:i + batch_size]
    print(f"Обработка строк {i+1} - {i+len(batch)}")

    # Обработка каждой строки в порции
    for j, row in batch.iterrows():
        sentence = row['Sentence']
        doc = nlp(sentence)

        tokens = [word.text for sent in doc.sentences for word in sent.words]
        lemmas = [word.lemma for sent in doc.sentences for word in sent.words]
        pos_tags = [word.upos for sent in doc.sentences for word in sent.words]

        # Добавляем результаты в DataFrame
        processed_data.at[j, 'Tokens'] = ' '.join(tokens)
        processed_data.at[j, 'Lemmas'] = ' '.join(lemmas)
        processed_data.at[j, 'POS'] = ' '.join(pos_tags)

Обработка строк 1 - 100
Обработка строк 101 - 200
Обработка строк 201 - 300
Обработка строк 301 - 400
Обработка строк 401 - 500
Обработка строк 501 - 600
Обработка строк 601 - 700
Обработка строк 701 - 800
Обработка строк 801 - 900
Обработка строк 901 - 1000
Обработка строк 1001 - 1100
Обработка строк 1101 - 1200
Обработка строк 1201 - 1300
Обработка строк 1301 - 1400
Обработка строк 1401 - 1500
Обработка строк 1501 - 1600
Обработка строк 1601 - 1700
Обработка строк 1701 - 1800
Обработка строк 1801 - 1900
Обработка строк 1901 - 2000
Обработка строк 2001 - 2089


In [6]:
 # Сохраняем результаты текущей порции в CSV
processed_data.to_csv(output_file_path, index=False, sep=';')
print(f"Сохранение результатов до строки {i+len(batch)} в {output_file_path}")

Сохранение результатов до строки 2089 в parsed_blocks_with_nlp.csv


In [7]:
print("Обработка завершена!")

Обработка завершена!
